In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew
import igraph
import os
pd.set_option('display.max_columns', 30)

# load data

In [2]:
ddir = "/home/scai/PhenPred/data/"
data_folder = "/home/scai/PhenPred/data/clines"

In [3]:
# Import samplesheets
cols = ["model_id", "BROAD_ID", "tissue", "cancer_type"]
col_rename = dict(
    ModelID="BROAD_ID",
    SangerModelID="model_id",
    SampleCollectionSite="tissue",
    OncotreeLineage="cancer_type",
)
ss_cmp = pd.read_csv(f"{data_folder}/model_list_20230505.csv")

ss_depmap = pd.read_csv(f"{data_folder}/depmap24Q4/Model.csv")
ss_depmap.rename(columns=col_rename, inplace=True)

# Map sample IDs to Sanger IDs
samplesheet = pd.concat(
    [
        ss_cmp[cols].dropna().assign(source="sanger"),
        ss_depmap[cols].dropna().assign(source="broad"),
    ]
)
samplesheet = samplesheet.groupby("model_id").first().reset_index()
samplesheet.replace(
    {
        "tissue": dict(
            large_intestine="Large Intestine",
            lung="Lung",
            ovary="Ovary",
            haematopoietic_and_lymphoid_tissue="Haematopoietic and Lymphoid",
            bone_marrow="Other tissue",
            upper_aerodigestive_tract="Other tissue",
            ascites="Other tissue",
            pleural_effusion="Other tissue",
        )
    },
    inplace=True,
)
tissue_map = samplesheet.set_index("model_id").to_dict()["tissue"]

# Growth
growth = pd.read_csv(f"{data_folder}/growth_rate_20220907.csv")
growth = (
    growth.sort_values(["model_id", "replicates"], ascending=False)
    .groupby("model_id")
    .first()
)
growth = growth.dropna(subset=["day4_day1_ratio"])

In [4]:
samplesheet.head()

,model_id,BROAD_ID,tissue,cancer_type,source
0,SIDM00001,ACH-000405,Haematopoietic and Lymphoid,Other Blood Cancers,sanger
1,SIDM00002,ACH-002340,Peripheral Nervous System,Neuroblastoma,sanger
2,SIDM00003,ACH-002159,Skin,Melanoma,sanger
3,SIDM00005,ACH-000044,Breast,Breast Carcinoma,sanger
4,SIDM00006,ACH-001552,Skin,Other Solid Cancers,sanger


In [5]:
samplesheet['tissue'].unique()

array(['Haematopoietic and Lymphoid', 'Peripheral Nervous System', 'Skin',
       'Breast', 'Ovary', 'Large Intestine', 'Esophagus', 'Lung',
       'Head and Neck', 'Central Nervous System', 'Kidney', 'Soft Tissue',
       'Bladder', 'Bone', 'Thyroid', 'Endometrium', 'Stomach', 'Pancreas',
       'Liver', 'Cervix', 'Eye', 'Prostate', 'Biliary Tract', 'Uterus',
       'Testis', 'Other tissue', 'Placenta', 'Small Intestine',
       'Adrenal Gland', 'Vulva', 'Unknown'], dtype=object)

In [6]:
PALETTE_TTYPE = {
    "Lung": "#007fff",
    "Prostate": "#665d1e",
    "Stomach": "#ffbf00",
    "Central Nervous System": "#fbceb1",
    "Skin": "#ff033e",
    "Bladder": "#ab274f",
    "Haematopoietic and Lymphoid": "#d5e6f7",
    "Kidney": "#7cb9e8",
    "Thyroid": "#efdecd",
    "Soft Tissue": "#8db600",
    "Head and Neck": "#e9d66b",
    "Ovary": "#b284be",
    "Bone": "#b2beb5",
    "Endometrium": "#10b36f",
    "Breast": "#6e7f80",
    "Pancreas": "#ff7e00",
    "Peripheral Nervous System": "#87a96b",
    "Cervix": "#c9ffe5",
    "Large Intestine": "#9f2b68",
    "Liver": "#00ffff",
    "Vulva": "#008000",
    "Esophagus": "#cd9575",
    "Biliary Tract": "#72a0c1",
    "Other tissue": "#a32638",
    "Small Intestine": "#9966cc",
    "Placenta": "#f19cbb",
    "Testis": "#e32636",
    "Adrenal Gland": "#3b7a57",
    "Uterus": "#7a3b5e",
    "Unknown": "#a32638",
    "Eye": "#ff1493",
}

In [7]:
# timestamp = "20250225_145621"
timestamp = "20250508_160635"
# Datasets - synthetic
## Transcriptomics
gexp_df = pd.read_csv(
    f"/home/scai/PhenPred/reports/vae/files/{timestamp}_imputed_transcriptomics.csv.gz",
    index_col=0,
)

## CRISPR-Cas9
cas9_df = pd.read_csv(
    f"/home/scai/PhenPred/reports/vae/files/{timestamp}_imputed_crisprcas9.csv.gz",
    index_col=0,
)

In [8]:
growth.shape

(964, 6)

In [9]:
gexp_measured = pd.read_csv(
    f"{data_folder}/depmap24Q4/OmicsExpressionGenesExpectedCountProfileVoom.csv",
    index_col=0,
).T
gexp_measured = gexp_measured.rename(
    index=samplesheet.reset_index().groupby("BROAD_ID").first()["model_id"]
)
gexp_measured = gexp_measured[gexp_measured.index.isin(gexp_df.index)]

In [10]:
## CRISPR-Cas9
cas9_measured = pd.read_csv(
    f"{data_folder}/depmap24Q4/CRISPRGeneEffect.csv", index_col=0
)
cas9_measured.columns = cas9_measured.columns.str.split(" ").str[0]
# cas9_measured = scale(cas9_measured.T).T
cas9_measured = cas9_measured.rename(
    index=samplesheet.reset_index().groupby("BROAD_ID").first()["model_id"]
)
cas9_measured = cas9_measured[cas9_measured.index.isin(cas9_df.index)]

In [11]:
measured_gexp_only = list(set(gexp_measured.index) - set(cas9_measured.index))
measured_cas9_only = list(set(cas9_measured.index) - set(gexp_measured.index))
measured_both = list(set(gexp_measured.index) & set(cas9_measured.index))
measured_no_cas9 = list(set(cas9_df.index) - set(cas9_measured.index))

In [12]:
measured_groups = {
    "both": measured_both,
    "gexp_only": measured_gexp_only,
    "cas9_only": measured_cas9_only,
    "none": [],  # Will be automatically assigned for remaining samples
}

In [13]:
df_res_vae_annot = pd.read_csv(
    f"../reports/vae/crispr/{timestamp}_transcriptomics_crisprcas9_remove_latent_n3_no_tissue_standardizedTrue_annot.csv.gz"
)
df_res_vae_annot["log10fdr_orig"] = -np.log10(df_res_vae_annot["fdr_orig"])
df_res_vae_annot["log10fdr_vae"] = -np.log10(df_res_vae_annot["fdr_vae"])
df_res_vae_annot["diff_log10fdr"] = (
    df_res_vae_annot["log10fdr_vae"] - df_res_vae_annot["log10fdr_orig"]
)

In [14]:
df_res_vae_annot_filtered = df_res_vae_annot.query(
    "entropy > 0.6 and fdr_vae < 0.05 and beta_vae > 0"
)
df_res_vae_annot_filtered = df_res_vae_annot_filtered[
    df_res_vae_annot_filtered["y_id"].isin(gexp_df.columns)
]
# Cap y_id rows to 3 per group
df_res_vae_annot_filtered_cap = (
    df_res_vae_annot_filtered.groupby("y_id").head(3).reset_index(drop=True)
)
df_res_vae_annot_filtered_cap_top = df_res_vae_annot_filtered_cap.head(2000)

In [15]:
df_res_vae_annot_filtered_cap_top.query("`y_id` == 'CNOT7'")

,y_id,x_id,n_orig,beta_orig,lr_orig,covs_orig,pval_orig,fdr_orig,n_vae,beta_vae,lr_vae,covs_vae,pval_vae,fdr_vae,skew_orig,skew_mosa,target_detailed,target,entropy,log10fdr_orig,log10fdr_vae,diff_log10fdr


In [16]:
df_res_vae_annot_filtered_cap_top['y_id'].unique()

array(['FAM50A', 'DDX3X', 'EIF1AX', ..., 'PDSS1', 'ATP2A2', 'RNF168'],
      dtype=object)

In [17]:
df_res_vae_annot_filtered_cap_top.head(20)

,y_id,x_id,n_orig,beta_orig,lr_orig,covs_orig,pval_orig,fdr_orig,n_vae,beta_vae,lr_vae,covs_vae,pval_vae,fdr_vae,skew_orig,skew_mosa,target_detailed,target,entropy,log10fdr_orig,log10fdr_vae,diff_log10fdr
0,FAM50A,FAM50B,923.0,0.75825,775.49438,208.0,1.148490e-170,1.975632e-166,1523.0,0.62305,846.16667,211.0,4.953857e-186,1.802213e-182,-0.62729,-0.38142,No link; CRISPR not in network,-,0.82245,165.704294,181.744194,16.039900
1,DDX3X,DDX3Y,923.0,0.70104,628.13085,208.0,1.274517e-138,2.192424e-134,1523.0,0.54253,592.67442,211.0,6.564286e-131,2.388087e-127,0.54274,-0.18181,3,3,0.83160,133.659076,126.621950,-7.037126
2,DDX3X,USP9Y,923.0,0.66805,569.93911,208.0,5.788442e-126,9.957278e-122,1523.0,0.50986,522.08826,211.0,1.486670e-115,5.408505e-112,0.54274,-0.18181,3,3,0.83160,121.001859,111.266923,-9.734937
3,DDX3X,UTY,923.0,0.67039,566.34511,208.0,3.502334e-125,6.024714e-121,1523.0,0.50949,523.46762,211.0,7.449375e-116,2.710083e-112,0.54274,-0.18181,3,3,0.83160,120.220064,111.567017,-8.653046
4,EIF1AX,EIF1AY,923.0,0.69643,563.47514,208.0,1.474560e-124,2.536539e-120,1523.0,0.54443,528.78237,211.0,5.198192e-117,1.891102e-113,0.61340,-0.24279,1,1,0.83172,119.595758,112.723285,-6.872473
5,RPP25L,RPP25,923.0,0.72759,538.28779,208.0,4.445512e-119,7.647170e-115,1523.0,0.56127,505.75143,211.0,5.328306e-112,1.938438e-108,-1.61835,-0.32212,1,1,0.82017,114.116499,107.712548,-6.403951
6,EIF1AX,KDM5D,923.0,0.65625,485.69119,208.0,1.234150e-107,1.061493e-103,1523.0,0.51644,468.16965,211.0,8.018027e-104,1.458479e-100,0.61340,-0.24279,2,2,0.83172,102.974083,99.836100,-3.137983
7,EIF1AX,DDX3Y,923.0,0.63376,439.11539,208.0,1.686445e-97,1.450512e-93,1523.0,0.50970,446.67194,211.0,3.822961e-99,6.953966e-96,0.61340,-0.24279,2,2,0.83172,92.838479,95.157767,2.319289
8,DNAJC19,DNAJC15,923.0,0.58210,379.34899,208.0,1.724632e-84,2.966712e-80,1523.0,0.45934,371.14446,211.0,1.054415e-82,3.835960e-79,-0.92817,-0.54211,2,2,0.72379,79.527725,78.416126,-1.111599
9,TTC7A,TTC7B,923.0,0.75893,354.78483,208.0,3.847638e-79,6.618707e-75,1523.0,0.57301,345.32267,211.0,4.422964e-77,1.609074e-73,-2.64586,-1.87939,No link; CRISPR not in network,-,0.60100,74.179227,72.793424,-1.385803


# selection algo

In [18]:
from cell_line_selection_updated import select_validation_cell_lines

In [19]:
results = select_validation_cell_lines(
    df_res_vae_annot_filtered_cap_top,  # Updated data source
    gexp_df,
    cas9_df,
    crispr_threshold_percentile=50,
    target_expression_threshold_percentile=50,
    biomarker_expression_threshold_percentile=50,
    create_visualizations=True,
    output_dir="./validation_results_3_criteria",
    tissue_map=tissue_map,
)

Starting optimized cell line selection pipeline with 3-criteria percentile-based filtering...
Using gene-specific percentile-based thresholds...
Threshold percentiles:
  CRISPR threshold: 50th percentile per target gene
  Target expression threshold: 50th percentile per target gene
  Biomarker expression threshold: 50th percentile per biomarker gene
Extracting gene pairs and weights...
Calculating validation scores with gene-specific percentile-based criteria...
Found 1523 cell lines common to both expression and CRISPR datasets
Processing 2000 gene pairs with 20 parallel jobs...
Applying gene-specific thresholds: CRISPR > 50th percentile per target gene AND Target Expression > 50th percentile per target gene AND Biomarker Expression > 50th percentile per biomarker gene
Score calculation completed in 36.45 seconds
2000 out of 2000 gene pairs have at least one cell line
that passes all three gene-specific thresholds (CRISPR > 50th percentile per target gene AND Target Expression > 50th 

Selecting cell lines: 100%|██████████| 6/6 [00:02<00:00,  2.02it/s, covered=107]


Cell line selection completed in 5.44 seconds
Selected 6 cell lines that collectively cover 979 of 2000 gene pairs (48.95%)
Evaluating cell line selection...

Total pipeline execution time: 42.59 seconds

----- RESULTS SUMMARY -----

RESULTS WITH GENE-SPECIFIC PERCENTILE THRESHOLDS:
CRISPR > 50th percentile per target gene AND
Target Expression > 50th percentile per target gene AND
Biomarker Expression > 50th percentile per biomarker gene

Selected Cell Lines:
1. SIDM00638 (Cervix) - Covers 243 gene pairs (12.15% of total)
2. SIDM01154 (Haematopoietic and Lymphoid) - Covers 223 gene pairs (11.15% of total)
3. SIDM00139 (Lung) - Covers 156 gene pairs (7.80% of total)
4. SIDM00487 (Large Intestine) - Covers 130 gene pairs (6.50% of total)
5. SIDM00653 (Lung) - Covers 120 gene pairs (6.00% of total)
6. SIDM00065 (Liver) - Covers 107 gene pairs (5.35% of total)

Overall Coverage: 48.95%
Top Pair Coverage: 75.00%
Weighted Coverage: 50.40%
Enhanced visualizations saved to ./validation_result

In [22]:
coverage_matrix_df = results["coverage_matrix"][results["selected_cell_lines"]]

In [26]:
coverage_matrix_df[coverage_matrix_df.sum(axis=1) > 0].to_csv(
    "./validation_results_3_criteria/coverage_matrix_df_filtered.csv"
)

In [21]:
# You can access the selected cell lines
selected_cell_lines = results["selected_cell_lines"]
print("Selected cell lines:", selected_cell_lines)

# Get the gene pairs that each cell line is best for demonstrating
for i, cell_line in enumerate(selected_cell_lines):
    examples = results["gene_pair_examples"][i]
    print(f"\nTop gene pairs for {cell_line}:")
    for cas9_gene, gexp_gene, score in examples[:10]:  # Show top 10
        print(f"  {cas9_gene} (CRISPR) - {gexp_gene} (Expr) - Score: {score:.4f}")

Selected cell lines: ['SIDM00638', 'SIDM01154', 'SIDM00139', 'SIDM00487', 'SIDM00653', 'SIDM00065']

Top gene pairs for SIDM00638:
  RPP25L (CRISPR) - RPP25 (Expr) - Score: 5.0000
  ATP6V0E1 (CRISPR) - ATP6V0E2 (Expr) - Score: 3.2813
  PRDX1 (CRISPR) - PRDX2 (Expr) - Score: 1.5060
  CEPT1 (CRISPR) - CHPT1 (Expr) - Score: 1.4168
  WDR77 (CRISPR) - MTAP (Expr) - Score: 1.3462
  UBE2S (CRISPR) - UBE2C (Expr) - Score: 1.3393
  POP7 (CRISPR) - RPP25 (Expr) - Score: 1.3239
  SLC16A1 (CRISPR) - SLC16A3 (Expr) - Score: 1.3130
  EMC4 (CRISPR) - EMC10 (Expr) - Score: 1.2812
  PTK2 (CRISPR) - PTK2B (Expr) - Score: 1.1335

Top gene pairs for SIDM01154:
  INTS6 (CRISPR) - INTS6L (Expr) - Score: 3.6993
  CSTF2 (CRISPR) - CSTF2T (Expr) - Score: 2.6854
  RPS4X (CRISPR) - RPS4Y1 (Expr) - Score: 1.7425
  TBL1XR1 (CRISPR) - TBL1X (Expr) - Score: 1.7279
  ZFX (CRISPR) - ZFY (Expr) - Score: 1.6764
  ZFX (CRISPR) - RPS4Y1 (Expr) - Score: 1.6464
  RPS4X (CRISPR) - ZFY (Expr) - Score: 1.6011
  ZFX (CRISPR) - 